# * Import : "AGG_PERF_NEWCO" to "AGG_PERF_NEWCO_SNAP"
    WHERE CENTER IN ('Revenue', 'Sales')
    AND NOT REGEXP_LIKE(METRIC_CD, '[0-9]C$|[0-9]H$|[0-9]MCOM$')
    AND TM_KEY_?

## Parameter

In [8]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

In [9]:
config = configparser.ConfigParser()
config.read('../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 14)

## ETL Process

### Step 1 : AGG_PERF_NEWCO_SNAP
    Delete -> Insert

In [ ]:
# Example

# data = dict(dept_id=280, dept_name="Facility")
# cursor.execute("""
#         insert into departments (department_id, department_name)
#         values (:dept_id, :dept_name)""", data)

In [ ]:
# Test

# query = """
#     SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY
# 	, CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_CD, METRIC_NAME, SEQ
# 	, ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM
# 	, AREA_TYPE, AREA_CD, AREA_NAME
# 	, ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACTUAL_AGG, TARGET_AGG, BASELINE_AGG
# 	, PPN_TM

#     FROM GEOSPCAPPO.AGG_PERF_NEWCO NOLOCK

#     WHERE CENTER IN ('Revenue', 'Sales')
#     AND NOT REGEXP_LIKE(METRIC_CD, '[0-9]C$|[0-9]H$|[0-9]MCOM$') --|[0-9]CORP$|[0-9]GEO$|[0-9]A[A-K]$
#     --AND TM_KEY_YR = (SELECT V_YR FROM W_PARAM)
#     AND TM_KEY_DAY = :tm_key_day
# """

In [116]:
# Import Data Source

# Input : 0 for dummy parameter
v_param = dict(yr=0, mth_start=202401, mth_end=202403, dt=0)
# v_param = dict(yr=0, mth=202404, day=0)

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')

with open('SQL/Import-AGG_PERF_NEWCO_SNAP.sql', 'r') as sql_file:
    queries = sql_file.read().split(';')
    query = queries[0].strip()
    sql_file.close()
    
# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
v_date = (curr_dt, )


try:
    print(f"\nProcessing...\n\n -> data as of (YR: {v_param['yr']}, MTH_Start: {v_param['mth_start']}, MTH_End: {v_param['mth_end']}, DT: {v_param['dt']})")

    # Create Dataframe
    src_cur.execute(query, v_param)

    print(f'\nGet rows...')
    rows = src_cur.fetchall()

    print(f'\nGet src_df...')
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n  -> df : {src_df.shape[0]} rows, {src_df.shape[1]} columns') 

    # Connect : AKPIPRD
    tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
    tgt_conn = oracledb.connect(tgt_dsn)
    print(f'\n{AKPIPRD_db} : Connected')
    tgt_cur = tgt_conn.cursor()

    # # Truncate
    # tgt_cur.execute("TRUNCATE TABLE AUTOKPI.AGG_PERF_NEWCO_SNAP")
    # print(f'\n  -> TRUNCATE : "AGG_PERF_NEWCO_SNAP" : Done !')

    # Delete
    tgt_cur.execute("""
        DELETE AUTOKPI.AGG_PERF_NEWCO_SNAP 
        WHERE TM_KEY_MTH BETWEEN :mth_start AND :mth_end
        """, (v_param['mth_start'], v_param['mth_end']))
    print(f'\n  -> DELETE : AGG_PERF_NEWCO_SNAP : Done !')
    # tgt_cur.execute("""
    #     DELETE AUTOKPI.AGG_PERF_NEWCO_SNAP 
    #     WHERE TM_KEY_MTH = :tm_key_mth
    #     """, (v_param['mth_end'], ))
    # print(f'\n  -> DELETE : AGG_PERF_NEWCO_SNAP : Done !')

    # Insert
    tgt_cur.executemany("""
        INSERT INTO AGG_PERF_NEWCO_SNAP 
        (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_CD, METRIC_NAME, SEQ, ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACTUAL_AGG, TARGET_AGG, BASELINE_AGG, PPN_TM) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27)
        """, rows)
    tgt_cur.close()
    tgt_conn.commit()
    print(f'\n  -> INSERT : "AGG_PERF_NEWCO_SNAP" : Done !')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    tgt_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    # job_end_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\nJob Done !!!')



Job Start... 2024-06-14, 18:26:22

TDMDBPR : Connected

Processing...

 -> data as of (YR: 0, MTH_Start: 202401, MTH_End: 202403, DT: 0)

 -> df : 7452566 rows, 27 columns

AKPIPRD : Connected

 -> DELETE : AGG_PERF_NEWCO_SNAP : Done !

 -> INSERT : "AGG_PERF_NEWCO_SNAP" : Done !

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


In [ ]:
# # Test sub scipts

# v_param = dict(yr=0, mth_start=202401, mth_end=202403, dt=0)
# v_param['mth_start'], v_param['mth_end']

# # Connect : AKPIPRD
# tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
# tgt_conn = oracledb.connect(tgt_dsn)
# print(f'\n{AKPIPRD_db} : Connected')
# tgt_cur = tgt_conn.cursor()

# # Delete
# tgt_cur.execute("""
#     DELETE AUTOKPI.AGG_PERF_NEWCO_SNAP 
#     WHERE TM_KEY_MTH BETWEEN :mth_start AND :mth_end
#     """, (v_param['mth_start'], v_param['mth_end']))
# print(f'\n -> DELETE : AGG_PERF_NEWCO_SNAP : Done !')

# # Insert
# # tgt_cur.executemany("""
# #     INSERT INTO AGG_PERF_NEWCO_SNAP 
# #     (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_CD, METRIC_NAME, SEQ, ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACTUAL_AGG, TARGET_AGG, BASELINE_AGG, PPN_TM) 
# #     VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27)
# #     """, rows)
# tgt_cur.close()
# # tgt_conn.commit()
# # print(f'\n -> INSERT : "AGG_PERF_NEWCO_SNAP" : Done !')

# tgt_conn.close()

In [117]:
src_df.tail()
# src_df.dtypes
# df[['PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates()

,TM_KEY_YR,TM_KEY_QTR,TM_KEY_MTH,TM_KEY_WK,TM_KEY_DAY,CENTER,PRODUCT_GRP,COMP_CD,METRIC_GRP,METRIC_CD,...,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,BASELINE_SNAP,ACTUAL_AGG,TARGET_AGG,BASELINE_AGG,PPN_TM
7452561,2024,20241,202403,2024010,20240304,Sales,TVS,TRUE,Sales,TB4R001000CORP,...,HH,910098,Yasothon,NaN,NaN,NaN,710.0,934.451613,0.0,2024-06-14 11:01:19
7452562,2024,20241,202403,2024010,20240304,Sales,TVS,TRUE,Sales,TB4R001000CORP,...,HH,910096,Surin,300.0,391.579355,0.0,3763.0,1566.317419,0.0,2024-06-14 11:01:19
7452563,2024,20241,202403,2024010,20240308,Sales,TVS,TRUE,Sales,TB4R001000CORP,...,HH,907017,"BKK : Don Mueang, Sai Mai, Lak Si",1800.0,450.911613,0.0,2555.0,2254.558065,0.0,2024-06-14 11:01:19
7452564,2024,20241,202403,2024010,20240309,Sales,TVS,TRUE,Sales,TB4R001000CORP,...,HH,904074,Phrae,NaN,NaN,NaN,502.0,246.134516,0.0,2024-06-14 11:01:19
7452565,2024,20241,202403,2024010,20240309,Sales,TVS,TRUE,Sales,TB4R001000CORP,...,HH,904076,Tak,NaN,NaN,NaN,1267.0,2133.677419,0.0,2024-06-14 11:01:19
